## warning : 환경 바꿔야 함.

conda activate pointnext

In [1]:
cd /data/wlsgur4011/part_assembly/src/pointnext

/data/wlsgur4011/part_assembly/src/pointnext


In [2]:
%matplotlib widget

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="5"
from jhutil import show_point_clouds
import torch
import numpy as np

# 2. S3DIS Segmentation

## import and config

In [4]:
import argparse
import yaml
import os
import logging
import numpy as npb
import csv
import wandb
import glob
from tqdm import tqdm
import torch
import torch.nn as nn
from torch import distributed as dist, multiprocessing as mp
from torch.utils.tensorboard import SummaryWriter
# from torch_scatter import scatter
from openpoints.utils import set_random_seed, save_checkpoint, load_checkpoint, resume_checkpoint, setup_logger_dist, \
    cal_model_parm_nums, Wandb, generate_exp_directory, resume_exp_directory, EasyConfig, dist_utils, find_free_port
from openpoints.utils import AverageMeter, ConfusionMatrix, get_mious
from openpoints.dataset import build_dataloader_from_cfg, get_features_by_keys, get_class_weights
from openpoints.dataset.data_util import voxelize
from openpoints.dataset.semantic_kitti.semantickitti import load_label_kitti, load_pc_kitti, remap_lut_read, remap_lut_write, get_semantickitti_file_list
from openpoints.transforms import build_transforms_from_cfg
from openpoints.optim import build_optimizer_from_cfg
from openpoints.scheduler import build_scheduler_from_cfg
from openpoints.loss import build_criterion_from_cfg
from openpoints.models import build_model_from_cfg
import warnings

[2023-05-27 22:03:12] [DEBU] Popen(['git', 'version'], cwd=/data/wlsgur4011/part_assembly/src/pointnext, universal_newlines=False, shell=None, istream=None)
[2023-05-27 22:03:12] [DEBU] Popen(['git', 'version'], cwd=/data/wlsgur4011/part_assembly/src/pointnext, universal_newlines=False, shell=None, istream=None)
[2023-05-27 22:03:12] [DEBU] Trying paths: ['/root/.docker/config.json', '/root/.dockercfg']
[2023-05-27 22:03:12] [DEBU] No config file found
[2023-05-27 22:03:14] [DEBU] Creating converter from 7 to 5
[2023-05-27 22:03:14] [DEBU] Creating converter from 5 to 7
[2023-05-27 22:03:14] [DEBU] Creating converter from 7 to 5
[2023-05-27 22:03:14] [DEBU] Creating converter from 5 to 7


In [5]:
import sys
sys.argv = ["examples/segmentation/main.py" ,"--cfg" ,"cfgs/part_assembly/pointnext-s.yaml"]


In [6]:
def get_config():
    parser = argparse.ArgumentParser('Scene segmentation training/testing')
    parser.add_argument('--cfg', type=str, required=True, help='config file')
    parser.add_argument('--profile', action='store_true', default=False, help='set to True to profile speed')
    args, opts = parser.parse_known_args()
    cfg = EasyConfig()
    cfg.load(args.cfg, recursive=True)
    cfg.update(opts)  # overwrite the default arguments in yml
    
    if cfg.seed is None:
        cfg.seed = np.random.randint(1, 10000)
    
    # init distributed env first, since logger depends on the dist info.
    cfg.rank, cfg.world_size, cfg.distributed, cfg.mp = dist_utils.get_dist_info(cfg)
    cfg.sync_bn = cfg.world_size > 1
    
    # init log dir
    cfg.task_name = args.cfg.split('.')[-2].split('/')[-2]  # task/dataset name, \eg s3dis, modelnet40_cls
    cfg.cfg_basename = args.cfg.split('.')[-2].split('/')[-1]  # cfg_basename, \eg pointnext-xl
    tags = [
        cfg.task_name,  # task name (the folder of name under ./cfgs
        cfg.mode,
        cfg.cfg_basename,  # cfg file name
        f'ngpus{cfg.world_size}',
        f'seed{cfg.seed}',
    ]
    opt_list = []  # for checking experiment configs from logging file
    for i, opt in enumerate(opts):
        if 'rank' not in opt and 'dir' not in opt and 'root' not in opt and 'pretrain' not in opt and 'path' not in opt and 'wandb' not in opt and '/' not in opt:
            opt_list.append(opt)
    cfg.root_dir = os.path.join(cfg.root_dir, cfg.task_name)
    cfg.opts = '-'.join(opt_list)
    
    cfg.is_training = cfg.mode not in ['test', 'testing', 'val', 'eval', 'evaluation']
    if cfg.mode in ['resume', 'val', 'test']:
        resume_exp_directory(cfg, pretrained_path=cfg.pretrained_path)
        cfg.wandb.tags = [cfg.mode]
    else:
        generate_exp_directory(cfg, tags, additional_id=os.environ.get('MASTER_PORT', None))
        cfg.wandb.tags = tags
    os.environ["JOB_LOG_DIR"] = cfg.log_dir
    cfg_path = os.path.join(cfg.run_dir, "cfg.yaml")
    with open(cfg_path, 'w') as f:
        yaml.dump(cfg, f, indent=2)
        os.system('cp %s %s' % (args.cfg, cfg.run_dir))
    cfg.cfg_path = cfg_path
    
    # wandb config
    cfg.wandb.name = cfg.run_name
    
    if cfg.model.get('in_channels', None) is None:
        cfg.model.in_channels = cfg.model.encoder_args.in_channels
        
    return cfg

cfg = get_config()
set_random_seed(cfg.seed + cfg.rank, deterministic=cfg.deterministic)
torch.backends.cudnn.enabled = True

launch mp with 1 GPUs, current rank: 0


In [7]:

import jhutil; jhutil.jhprint(1111, cfg)

1111  {
    "dist_url": "tcp://localhost:8888",
    "dist_backend": "nccl",
    "multiprocessing_distributed": false,
    "ngpus_per_node": 1,
    "world_size": 1,
    "launcher": "mp",
    "local_rank": 0,
    "use_gpu": true,
    "seed": 7915,
    "epoch": 0,
    "epochs": 100,
    "ignore_index": null,
    "val_fn": "validate",
    "deterministic": false,
    "sync_bn": false,
    "criterion_args": {
        "NAME": "CrossEntropy",
        "label_smoothing": 0.2
    },
    "use_mask": false,
    "grad_norm_clip": 10,
    "layer_decay": 0,
    "step_per_update": 1,
    "start_epoch": 1,
    "sched_on_epoch": true,
    "wandb": {
        "use_wandb": false,
        "project": "PointNeXt-S3DIS",
        "tags": [
            "part_assembly",
            "train",
            "pointnext-s",
            "ngpus1",
            "seed7915"
        ],
        "name": "part_assembly-train-pointnext-s-ngpus1-seed7915-20230527-220320-JEYoxiby5nqSoGcgqYyNnB"
    },
    "use_amp": false,
    "use_v

## model & data_loader

In [8]:
model = build_model_from_cfg(cfg.model).to(cfg.rank)
model_size = cal_model_parm_nums(model)

[2023-05-27 22:03:20] [INFO] radius: [[0.1], [0.1], [0.2], [0.4], [0.8]],
 nsample: [[32], [32], [32], [32], [32]]
[2023-05-27 22:03:20] [INFO] NAME: ballquery
normalize_dp: True
radius: 0.1
nsample: 32
[2023-05-27 22:03:20] [INFO] NAME: ballquery
normalize_dp: True
radius: 0.2
nsample: 32
[2023-05-27 22:03:20] [INFO] NAME: ballquery
normalize_dp: True
radius: 0.4
nsample: 32
[2023-05-27 22:03:20] [INFO] NAME: ballquery
normalize_dp: True
radius: 0.8
nsample: 32


In [9]:
val_loader = build_dataloader_from_cfg(cfg.get('val_batch_size', cfg.batch_size),
                                       cfg.dataset,
                                       cfg.dataloader,
                                       datatransforms_cfg=cfg.datatransforms,
                                       split='val',
                                       distributed=cfg.distributed
                                    )

[2023-05-27 22:03:30] [INFO] 
Totally 17686 samples in val set


In [10]:
for idx, data in enumerate(val_loader):
    break

import jhutil; jhutil.jhprint(1111, data)

1111  {
    "pos": "tensor[1, 29751, 3] n=89253 (0.3Mb) x∈[-1.912, 5.612] μ=0.942 σ=1.797",
    "x": "tensor[1, 29751, 3] n=89253 (0.3Mb) x∈[-8.267, 2.856] μ=-2.607 σ=3.111",
    "y": "tensor[1, 29751] i64 0.2Mb x∈[0, 1] μ=0.137 σ=0.344",
    "heights": "tensor[1, 29751, 1] 0.1Mb x∈[0., 5.612] μ=2.827 σ=1.612"
}


## RUN !!!

In [11]:
val_loader.dataset[5041]

{'pos': tensor[53, 3] n=159 x∈[-0.091, 0.429] μ=0.092 σ=0.150,
 'x': tensor[53, 3] n=159 x∈[-7.436, 2.107] μ=-2.051 σ=3.078,
 'y': tensor[53] i64 x∈[1, 1] μ=1.000 σ=0.,
 'heights': tensor[53, 1] x∈[0.002, 0.431] μ=0.279 σ=0.117}

In [13]:
cfg.num_classes, cfg.ignore_index

(2, None)

In [17]:
model.eval()  # set model to eval mode
cm = ConfusionMatrix(num_classes=cfg.num_classes, ignore_index=cfg.ignore_index)

import jhutil; jhutil.jhprint(1111, cm.virtual_num_classes, cm.value)
pbar = tqdm(enumerate(val_loader), total=val_loader.__len__(), desc='Val')
for idx, data in pbar:
    keys = data.keys() if callable(data.keys) else data.keys
    for key in keys:
        data[key] = data[key].cuda(non_blocking=True)
    target = data['y'].squeeze(-1)
    if len(target) < 256:
        continue
    x = data['x']
    data['x'] = get_features_by_keys(data, cfg.feature_keys)
    

    # import jhutil; jhutil.jhprint(1111, data)
    logits = model(data)

    # import jhutil; jhutil.jhprint(2222, logits)
    # import jhutil; jhutil.jhprint(3333, target)
    
    # break
    

1111  "2" "0"


AttributeError: 'int' object has no attribute 'diag'

In [21]:

cm = ConfusionMatrix(num_classes=cfg.num_classes, ignore_index=cfg.ignore_index)
pbar = tqdm(enumerate(val_loader), total=val_loader.__len__(), desc='Val')
for idx, data in pbar:
    keys = data.keys() if callable(data.keys) else data.keys
    for key in keys:
        data[key] = data[key].cuda(non_blocking=True)
    target = data['y'].squeeze(-1)
    # if len(target) < 256:
    #     continue
    data['x'] = get_features_by_keys(data, cfg.feature_keys)
    logits = model(data)
    if 'mask' not in cfg.criterion_args.NAME or cfg.get('use_maks', False):
        cm.update(logits.argmax(dim=1), target)
    else:
        mask = data['mask'].bool()
        cm.update(logits.argmax(dim=1)[mask], target[mask])
    break


Val:   0%|          | 0/17686 [00:00<?, ?it/s]


In [22]:
cm.tp
cm.union
cm.count

tensor[2] i64 μ=1.488e+04 σ=1.527e+04 cuda:0 [25672, 4079]

In [ ]:
# x = data['x'][0].transpose(0, 1)[:, :3].cpu()

show_point_clouds(data["pos"].cpu())

In [ ]:
print(1111)

# 1. ShapeNet Part Segmentation

### import and config

In [ ]:
import argparse
import yaml
import os
import sys
import logging
import wandb
from tqdm import tqdm
import torch
import torch.nn as nn
from torch import distributed as dist, multiprocessing as mp
from torch.utils.tensorboard import SummaryWriter
from torch_scatter import scatter
import torch.nn.functional as F
import warnings
import numpy as np
from sklearn.metrics import confusion_matrix
from collections import defaultdict, Counter

torch.backends.cudnn.benchmark = False
warnings.simplefilter(action='ignore', category=FutureWarning)
sys.path.insert(0, "/data/wlsgur4011/part_assembly/src/pointnext/examples")

from openpoints.models import build_model_from_cfg
from openpoints.models.layers import torch_grouping_operation, knn_point
from openpoints.loss import build_criterion_from_cfg
from openpoints.scheduler import build_scheduler_from_cfg
from openpoints.optim import build_optimizer_from_cfg
from openpoints.dataset import build_dataloader_from_cfg, get_class_weights, get_features_by_keys
from openpoints.transforms import build_transforms_from_cfg
from openpoints.utils import AverageMeter, ConfusionMatrix
from openpoints.utils import set_random_seed, save_checkpoint, load_checkpoint, resume_checkpoint, setup_logger_dist, \
    cal_model_parm_nums, Wandb, generate_exp_directory, resume_exp_directory, EasyConfig, dist_utils, find_free_port
from openpoints.models.layers import furthest_point_sample



In [ ]:
import sys
sys.argv = ["examples/segmentation/main.py" ,"--cfg" ,"cfgs/shapenetpart/pointnext-s.yaml"]

In [ ]:
def get_config():
    parser = argparse.ArgumentParser('ShapeNetPart Part segmentation training')
    parser.add_argument('--cfg', type=str, required=True, help='config file')
    args, opts = parser.parse_known_args()
    cfg = EasyConfig()
    cfg.load(args.cfg, recursive=True)
    cfg.update(opts)
    if cfg.seed is None:
        cfg.seed = np.random.randint(1, 10000)
    # init distributed env first, since logger depends on the dist info.
    cfg.rank, cfg.world_size, cfg.distributed, cfg.mp = dist_utils.get_dist_info(cfg)
    cfg.sync_bn = cfg.world_size > 1

    # logger
    # init log dir
    cfg.task_name = args.cfg.split('.')[-2].split('/')[-2]  # task/dataset name, \eg s3dis, modelnet40_cls
    cfg.cfg_basename = args.cfg.split('.')[-2].split('/')[-1]  # cfg_basename, \eg pointnext-xl
    tags = [
        cfg.task_name,  # task name (the folder of name under ./cfgs
        cfg.mode,
        cfg.cfg_basename,  # cfg file name
        f'ngpus{cfg.world_size}',
        f'seed{cfg.seed}',
    ]
    opt_list = [] # for checking experiment configs from logging file
    for i, opt in enumerate(opts):
        if 'rank' not in opt and 'dir' not in opt and 'root' not in opt and 'pretrain' not in opt and 'path' not in opt and 'wandb' not in opt and '/' not in opt:
            opt_list.append(opt)
    cfg.root_dir = os.path.join(cfg.root_dir, cfg.task_name)
    cfg.opts = '-'.join(opt_list)

    cfg.is_training = cfg.mode not in ['test', 'testing', 'val', 'eval', 'evaluation']

    if cfg.mode in ['resume', 'test', 'val']:
        resume_exp_directory(cfg, pretrained_path=cfg.pretrained_path)
        cfg.wandb.tags = [cfg.mode]
    else:
        generate_exp_directory(cfg, tags, additional_id=os.environ.get('MASTER_PORT', None))
        cfg.wandb.tags = tags
    os.environ["JOB_LOG_DIR"] = cfg.log_dir
    cfg_path = os.path.join(cfg.run_dir, "cfg.yaml")
    with open(cfg_path, 'w') as f:
        yaml.dump(cfg, f, indent=2)
        os.system('cp %s %s' % (args.cfg, cfg.run_dir))
    cfg.cfg_path = cfg_path
    return cfg

cfg = get_config()

### dataloader and model

In [ ]:

val_loader = build_dataloader_from_cfg(cfg.batch_size,
                                       cfg.dataset,
                                       cfg.dataloader,
                                       datatransforms_cfg=cfg.datatransforms,
                                       split='val',
                                       distributed=False
                                       )
logging.info(f"length of validation dataset: {len(val_loader.dataset)}")
num_classes = val_loader.dataset.num_classes if hasattr(
    val_loader.dataset, 'num_classes') else None
if num_classes is not None:
    assert cfg.num_classes == num_classes

cfg.cls2parts = val_loader.dataset.cls2parts

if cfg.model.get('decoder_args', False):
    cfg.model.decoder_args.cls2partembed = val_loader.dataset.cls2partembed
if cfg.model.get('in_channels', None) is None:
    cfg.model.in_channels = cfg.model.encoder_args.in_channels
    
model = build_model_from_cfg(cfg.model).cuda()

In [ ]:
train_loader = build_dataloader_from_cfg(cfg.batch_size,
                                         cfg.dataset,
                                         cfg.dataloader,
                                         datatransforms_cfg=cfg.datatransforms,
                                         split='train',
                                         distributed=False,
                                         )
len(train_loader.dataset)

### RUN!!! 

In [ ]:

from openpoints.dataset import get_features_by_keys
from openpoints.loss import build_criterion_from_cfg


In [ ]:
criterion = build_criterion_from_cfg(cfg.criterion_args).cuda()
for data in train_loader:
    import jhutil; jhutil.jhprint(1000, data)
    batch_size, num_point, _ = data['pos'].size()
    for key in data.keys():
        data[key] = data[key].cuda(non_blocking=True)
    target = data['y']
    data['x'] = get_features_by_keys(data, cfg.feature_keys)
    import jhutil; jhutil.jhprint(1111, data['x'])
    import jhutil; jhutil.jhprint(1222, data['y'])

    logits = model(data)
    import jhutil; jhutil.jhprint(2222, logits)
    if cfg.criterion_args.NAME != 'MultiShapeCrossEntropy':
        loss = criterion(logits, target)
        import jhutil; jhutil.jhprint(3111, )
    else:
        loss = criterion(logits, target, data['cls'])
        import jhutil; jhutil.jhprint(3222, )
    import jhutil; jhutil.jhprint(3333, loss)
    
    break

In [ ]:

import jhutil; jhutil.jhprint(1111, data['x'][0, :,::200].v) # continuous한 feature
import jhutil; jhutil.jhprint(1222, data['y'][0, ::200].v)   # label 값

In [ ]:
import jhutil; jhutil.jhprint(6666, "feature_keys:", cfg.feature_keys) # 그냥  coordinate 이외에 다른 값들을 주나 봄

In [ ]:
x = data['x'][0].transpose(0, 1)[:, 3:6].cpu()
pos = data['pos'][0].cpu()

In [ ]:
show_point_clouds([x, pos])